In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import json

from oraclai.llm_guide.utils import *
from oraclai.llm_guide.init_utils import *
from oraclai.llm_guide.infer_utils import *
from oraclai.llm_guide.loop_utils import *
from oraclai.llm_guide.mongo_utils import *

In [3]:
load_dotenv()

True

In [4]:
os.getenv("APP_NAME")

'ADMIN'

In [5]:
action_func_db.delete_many({ 'app': os.getenv("APP_NAME") })
func_db.delete_many({ 'app': os.getenv("APP_NAME") })

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff0000000000000389'), 'opTime': {'ts': Timestamp(1722295435, 10), 't': 905}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1722295435, 10), 'signature': {'hash': b'\xe1\r!\x92\x93\x13$G\xc0\xce\xfe\x0fa:^:U]d\xd5', 'keyId': 7363753538167504968}}, 'operationTime': Timestamp(1722295435, 10)}, acknowledged=True)

In [16]:
model = json.load(open(f'./report/{os.getenv("APP_NAME").split('-')[0]}.json'))

In [17]:
queue = [list(model['nodes'].keys())[0]]

In [18]:
model['nodes'][queue[0]]['depth'] = 0
model['nodes'][queue[0]]['prev_action'] = None
model['nodes'][queue[0]]['prev_state'] = None

In [20]:
for state_id, edges in model['edges'].items():
    for action_id, target_state_id in edges.items():
        # print(state_id, target_state_id)
        model['nodes'][target_state_id]['prev_state'] = state_id
        # print(action_id)
        model['nodes'][target_state_id]['prev_action'] = list(filter(
            lambda x: x['id'] == action_id,
            model['nodes'][state_id]['actions']
        ))[0]
        model['nodes'][target_state_id]['depth'] = model['nodes'][state_id]['depth'] + 1

In [21]:
ACTION_COUNTER = 0

In [23]:
while len(queue) > 0:
    state_id = queue[0]
    logger.info(f"Visiting state {state_id}")
    
    state = model['nodes'][state_id]

    for action in state['actions']:
        ACTION_COUNTER += 1
        logger.info(f'Extracting action scenarios: {action['id']}')

        # if action does not lead to a new state ignore it
        if (state_id not in model['edges'] or action['id'] not in model['edges'][state_id]) and \
            ('isCritical' not in action or not action['isCritical']):
            continue
        
        functionalities = extract_action_functionalities_dict(state, action)
        
        if len(functionalities) != 0:
            functionality_ids = insert_functionalities(functionalities)
            
            insert_action_functionality(
                func_ids=functionality_ids,
                state_id=state_id,
                state_url=state['url'],
                prev_state_id=state['prev_state'],
                action_id=action['id'],
                prev_action_id=state['prev_action'] if state['prev_action'] is not None else None,
                action_test_id=action['testId'],
                action_depth=state['depth'],
                action_type="SINGLE"
            )
    
        if state['depth'] > 0:
            logger.info('Extracting double action scenarios')
            
            functionalities = extract_action_functionalities_dict(state, action, state['prev_action'])
            
            if len(functionalities) != 0:
                functionality_ids = insert_functionalities(functionalities)
                insert_action_functionality(
                    func_ids=functionality_ids,
                    state_id=state_id,
                    state_url=state['url'],
                    prev_state_id=state['prev_state'],
                    action_id=action['id'],
                    prev_action_id=state['prev_action'] if state['prev_action'] is not None else None,
                    action_test_id=action['testId'],
                    action_depth=state['depth'],
                    action_type="DOUBLE"
                )
            
            logger.info('Updating action scores')

            update_functionality_score_dict(
                { 'id': state['prev_state'], ** model['nodes'][state['prev_state']]},
                state['prev_action'],
                { 'id': state_id, **state },
                action
            )

            logger.info('Action scores updated')

        logger.info('Marking final functionalities')

        mark_final_functionalities_dict({ 'id': state_id, **state }, action)

        logger.info('Final actions marked')

    if state_id in model['edges']:
        for target_state in model['edges'][state_id].values():
            queue.append(target_state)

    queue = queue[1:]

  Visiting state f3da9e519a1a495ba0b9e0d8501cc460495b087624abd9f91180ebf647b3126b
  Extracting action scenarios: dashboard-app
  Extracting action scenarios: //BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/ASIDE[1]/DIV[2]/OL[1]/LI[1]
  Extracting action scenarios: //BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/ASIDE[1]/DIV[2]/OL[1]/LI[1]/A[1]
  Extracting action scenarios: //BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/ASIDE[1]/DIV[2]/OL[1]/UL[1]/DIV[1]/LI[1]/DIV[1]
  Prompt:
  {"webpage_context": "A dashboard interface for managing an online store, displaying sales metrics, product information, and various administrative options.", "action_element": "<div class=\"_18fs8ps16u _18fs8ps19f\"><a class=\"_18fs8ps16u _18fs8psf\" href=\"/dashboard/products/\"><div class=\"_18fs8psmx _18fs8pspc _18fs8ps1gf _18fs8ps1fi _18fs8ps1i9 _18fs8ps1hc _18fs8ps3 _18fs8ps10 _18fs8psdr\"><svg class=\"yswmh70 yswmh72 _18fs8ps2ad\"><path></path></svg><span class=\"_1ruekcrq _1ruekcrv _1ruekcrz _1ruekcr15 _18fs8ps1k6 _18